In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
tokenizer = AutoTokenizer.from_pretrained("MohammedDhiyaEddine/job-skill-sentence-transformer-tsdae")
model = AutoModel.from_pretrained("MohammedDhiyaEddine/job-skill-sentence-transformer-tsdae").to('cuda')

In [ ]:
# df=pd.read_csv('/kaggle/input/job-recommendation-case-study/jobs.tsv', sep='\t')
df = pd.read_csv('/kaggle/input/job-recommendation-case-study/jobs.tsv',sep = '\t',usecols=range(3,4)).astype(str)#.head(100)
df['Cleaned_Description'] = ''
df.head()
df
# del df

In [ ]:
import re
from bs4 import BeautifulSoup
def cleanResume(resumeText):
    soup=BeautifulSoup(resumeText,"html.parser")
    resumeText=soup.get_text()
    resumeText = re.sub('http\S+\s*', ' ', resumeText)  # remove URLs
    resumeText = re.sub('RT|cc', ' ', resumeText)  # remove RT and cc
    resumeText = re.sub('#\S+', '', resumeText)  # remove hashtags
    resumeText = re.sub('@\S+', '  ', resumeText)  # remove mentions
    resumeText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', resumeText)  # remove punctuations
    resumeText = re.sub(r'[^\x00-\x7f]',r' ', resumeText) 
    resumeText = re.sub('\s+', ' ', resumeText)  # remove extra whitespace
    return resumeText
    
df['Cleaned_Description'] = df.Description.apply(lambda x: cleanResume(x))
df


In [ ]:
df['Source'] = 'KaggleManda'
df.drop(["Description"],inplace=True,axis=1)
# df = df.rename(columns={'old_name': 'new_name'})
df# assign values to new column

In [ ]:
df.to_csv("KaggleManda.csv",index=False)

In [ ]:
# del df
df=pd.read_csv("/kaggle/input/scrapeddata/cleaned_jobs.csv").drop(["Unnamed: 0","Location","Company","Position"],axis=1)
# df=pd.read_csv("/kaggle/input/kagglemanda/KaggleManda_embeddings.csv").drop(["Source"],axis=1).dropna()
# df = df.rename(columns={'Cleaned_Resume': 'Cleaned_Description'})
df

In [ ]:
# df2.shape

In [ ]:
# merged_df =merged_df = pd.concat([df1, df2])
# merged_df.shape

In [ ]:
# merged_df.to_csv("Merged.csv",index=False)

In [ ]:
df=pd.read_csv("/kaggle/input/merged/Merged.csv").drop(["Source"],axis=1).dropna()
df

In [ ]:
df.iloc[22603]

In [ ]:
list_of_lists =[item for sublist in df.values.tolist() for item in sublist]
len(list_of_lists)

In [ ]:
# print(len(temp))

In [ ]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
import numpy as np
n=int(df.shape[0])
max_len=512
embeddings=torch.Tensor([]).to('cuda')#(np.zeros((n,df.shape[1],max_len,768)))
for i in range(n):
    sentences =list_of_lists[i]
    encoded_sentences = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt",max_length=max_len)
#     encoded_sentences = tokenizer(sentences, return_tensors="pt")
    encoded_sentences.to('cuda')
#     print(len(encoded_sentences))
#     print(i)
#     print("encoded_sentences size=",len(encoded_sentences[0]),len(encoded_sentences['input_ids']),len(encoded_sentences['input_ids']))
    with torch.no_grad():
#         temp=[]
#         print(model(encoded_sentences['input_ids'], attention_mask=encoded_sentences['attention_mask'])[0])
        temp=model(**encoded_sentences)
#     print(model(**encoded_sentences)[0])
    sentence_embeddings = mean_pooling(temp, encoded_sentences['attention_mask'])
#         print(temp)
#     embeddings[i]=temp
#     embeddings.append(temp)
    embeddings=torch.cat((embeddings,sentence_embeddings),0)

In [ ]:
print(len(embeddings))
print(len(embeddings[0]))
# print(len(embeddings[0][0]))
# print(len(embeddings[0][0][0]))

In [ ]:
print(len(embeddings[0][0][0]))

In [ ]:
embeddings

In [ ]:
import pickle
# Open a file for writing
with open('jobEmbeddingArya.pickle', 'wb') as f:
    # Use pickle to dump the variable to the file
    pickle.dump(embeddings, f)

In [ ]:
import pickle

# Open the pickle file for reading
with open('jobEmbeddingArya.pickle', 'rb') as f:
#     Use pickle to load the variable from the file
    embeddings = pickle.load(f)

# Print the loaded variable
print(embeddings)


In [ ]:
# list_of_lists[22603]

In [ ]:
# # for my_string in list_of_lists[0]:
# #         words = my_string.split()
# # len(words)
# list1=[]
# list1.append(list_of_lists[22603].split())
# # list1.append(list_of_lists[1].split())
# list1

In [ ]:
list1=[]
for i in range(len(list_of_lists)):
#         words = my_string.split()
#         print(my_string)
#     if(i>22600 and i<22700):
#         print(i)
    list1.append(list_of_lists[i].split())
len(list1)

In [ ]:
corpus=[item for sublist in list1 for item in sublist]
len(corpus)

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
  
stop_words = set(stopwords.words('english'))
filtered_sentence = [w for w in list1 if not w.lower() in stop_words]
len(filtered_sentence)

In [ ]:
from gensim.models import Word2Vec
# my_string = "This is a string with spaces"
# list1=[item for sublist in list1 for item in sublist]
# Train a Word2Vec model on the sentences
model = Word2Vec(vector_size=300,window=5,min_count=5,workers=-1)
model.build_vocab(corpus)
model.train(corpus,total_examples=model.corpus_count,epochs=5)
model.save("word2vecArya.model")
model = Word2Vec.load("word2vecArya.model")
# Get the w2v representation of a sentence
# sentence = ['this', 'is', 'sentence', 'one']
# w2v_representation = model.wv[list_of_lists[0][0]]
# for word in list1:
#   vector = model.wv[word]

# print(vector)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(analyzer='word',ngram_range=(1,2),min_df=25,stop_words="english")
tfidf.fit(df['clean'])
tfidf_list=dict(zip(tfidf.get_feature_names_out(),list(tfidf.idf_)))
tfidf_feature=tfidf.get_feature_names_out()

In [ ]:
import numpy as np
from tqdm import tqdm
tfidf_vectors=[]
line=0
for desc in tqdm(corpus):
    sent_vec=np.zeros(300)
    weight_sum=0
    for word in desc:
        if word in model.wv.key_to_index and word in tfidf_feature:
            vec=model.wv[word]
            tf_idf=tfidf_list[word]*(desc.count(word)/len(desc))
            sent_vec+=(vec*tf_idf)
            weight_sum+=tf_idf
        if weight_sum!=0:
            sent_vec/=weight_sum
        tfidf_vectors.append(sent_vec)
        line+=1

In [ ]:
len(tfidf_vectors)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
# cosine_similarity=cosine_similarity(tfidf_vectors,tfidf_vectors)

## Dealing with user_embedding

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
tokenizer = AutoTokenizer.from_pretrained("MohammedDhiyaEddine/job-skill-sentence-transformer-tsdae")
model = AutoModel.from_pretrained("MohammedDhiyaEddine/job-skill-sentence-transformer-tsdae").to('cuda')

In [ ]:
user_df=pd.read_csv("/kaggle/input/userresume/UserResume.csv").dropna().drop(["Resume"],axis=1)
# user_df["Combined"] = df["Category"].astype(str) + df["Cleaned_Resume"]
user_df.head()

In [ ]:
user_df["Combined"] = user_df["Category"].astype(str) + user_df["Cleaned_Resume"]
user_df=user_df.drop(["Category","Cleaned_Resume"],axis=1)
user_df

In [ ]:
list_of_lists =[item for sublist in user_df.values.tolist() for item in sublist]
len(list_of_lists)

In [ ]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
import numpy as np
n=int(user_df.shape[0])
max_len=512
embeddings=torch.Tensor([]).to('cuda')#(np.zeros((n,df.shape[1],max_len,768)))
for i in range(n):
    sentences =list_of_lists[i]
    encoded_sentences = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt",max_length=max_len)
    encoded_sentences.to('cuda')
    with torch.no_grad():
        temp=model(**encoded_sentences)
    sentence_embeddings = mean_pooling(temp, encoded_sentences['attention_mask'])
    embeddings=torch.cat((embeddings,sentence_embeddings),0)

In [ ]:
import pickle
# Open a file for writing
with open('userEmbeddingArya.pickle', 'wb') as f:
    # Use pickle to dump the variable to the file
    pickle.dump(embeddings, f)

In [ ]:
import pickle

# Open the pickle file for reading
with open('userEmbeddingArya.pickle', 'rb') as f:
    # Use pickle to load the variable from the file
    embeddings = pickle.load(f)

# Print the loaded variable
print(len(embeddings),len(embeddings[0]))


## Stacking user and job embeddings

In [ ]:
import pickle

# Open the pickle file for reading
with open('/kaggle/input/embeddings/userEmbeddingArya.pickle', 'rb') as f:
    # Use pickle to load the variable from the file
    user_embeddings = pickle.load(f)

with open('/kaggle/input/embeddings/jobEmbeddingArya.pickle', 'rb') as f:
    # Use pickle to load the variable from the file
    job_embeddings = pickle.load(f)
    
# Print the loaded variable
print(len(user_embeddings),len(user_embeddings[0]))
print(len(job_embeddings),len(job_embeddings[0]))

In [ ]:
import itertools
import pandas as pd

# Create two example tensors
tensor1 = user_embeddings
tensor2 = job_embeddings

# Generate all permutations of the two tensors
permutations = itertools.product(tensor1, tensor2)

# Convert tuples to rows in a DataFrame
Embedding_df = pd.DataFrame(permutations, columns=['tensor1', 'tensor2'])

# Display the resulting DataFrame
print(Embedding_df.shape)


In [ ]:
Embedding_df.shape

In [ ]:
import pickle
# Open a file for writing
with open('Embedding_df.pickle', 'wb') as f:
    # Use pickle to dump the variable to the file
    pickle.dump(Embedding_df, f)

In [ ]:
# import torch.nn.functional as F
# Embedding_df['similarity'] = F.cosine_similarity(Embedding_df['tensor1'], Embedding_df['tensor2'])

### Creating Cosine similairty matrix

In [4]:
import pickle
import torch
import numpy as np
# Open the pickle file for reading
with open('/kaggle/input/embedding-df/Embedding_df.pickle', 'rb') as f:
    # Use pickle to load the variable from the file
    Embedding_df = pickle.load(f)
Embedding_df.head()

,tensor1,tensor2
0,"[tensor(-0.4269, device='cuda:0'), tensor(0.22...","[tensor(-0.1817, device='cuda:0'), tensor(0.29..."
1,"[tensor(-0.4269, device='cuda:0'), tensor(0.22...","[tensor(-0.0982, device='cuda:0'), tensor(0.23..."
2,"[tensor(-0.4269, device='cuda:0'), tensor(0.22...","[tensor(-0.3339, device='cuda:0'), tensor(0.22..."
3,"[tensor(-0.4269, device='cuda:0'), tensor(0.22...","[tensor(-0.1576, device='cuda:0'), tensor(0.15..."
4,"[tensor(-0.4269, device='cuda:0'), tensor(0.22...","[tensor(-0.1543, device='cuda:0'), tensor(0.32..."


In [6]:
import torch.nn.functional as F
Embedding_df['cosine_similarity']=np.zeros((Embedding_df.shape[0]))

for i in range(Embedding_df.shape[0]):
    tensor1 = Embedding_df.iloc[i]['tensor1']
    tensor2 = Embedding_df.iloc[i]['tensor2']
# compute the cosine similarity between the two tensors
    cosine_similarity_value = F.cosine_similarity(tensor1.unsqueeze(0), tensor2.unsqueeze(0)).item()
    Embedding_df['cosine_similarity']=cosine_similarity_value
Embedding_df.head()

,tensor1,tensor2,cosine_similarity
0,"[tensor(-0.4269, device='cuda:0'), tensor(0.22...","[tensor(-0.1817, device='cuda:0'), tensor(0.29...",0.768445
1,"[tensor(-0.4269, device='cuda:0'), tensor(0.22...","[tensor(-0.0982, device='cuda:0'), tensor(0.23...",0.768445
2,"[tensor(-0.4269, device='cuda:0'), tensor(0.22...","[tensor(-0.3339, device='cuda:0'), tensor(0.22...",0.768445
3,"[tensor(-0.4269, device='cuda:0'), tensor(0.22...","[tensor(-0.1576, device='cuda:0'), tensor(0.15...",0.768445
4,"[tensor(-0.4269, device='cuda:0'), tensor(0.22...","[tensor(-0.1543, device='cuda:0'), tensor(0.32...",0.768445


In [7]:
import pickle
# Open a file for writing
with open('Cosine.pickle', 'wb') as f:
    # Use pickle to dump the variable to the file
    pickle.dump(Embedding_df, f)

In [20]:

import pickle

# Open the pickle file for reading
with open('Cosine.pickle', 'rb') as f:
    # Use pickle to load the variable from the file
    Embedding_df = pickle.load(f)
Embedding_df.head()

RuntimeError: CUDA error: invalid argument
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import pickle

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(768*2, 512)
        # nn.relu()
        self.fc2 = nn.Linear(512, 256)
        # nn.relu()
        self.fc3 = nn.Linear(256, 128)
        # nn.relu()
        self.fc4 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.relu(x)
        x = self.fc4(x)
        return x
model = Model().cuda()
# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.01)


In [ ]:
y = torch.tensor(df['cosine_similarity'].values).to("cuda")


for epoch in range(10):
    for i in range(len(Embedding_df)/10000):
        optimizer.zero_grad()
        output = model(torch.cat((Embedding_df['tensor1'].values, Embedding_df['tensor2'].values), dim=1))
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()

    if epoch % 1 == 0:
        print('Epoch [%d/%d], Loss: %.4f' % (epoch+1, 100, loss.item()))